In [ ]:
import gaps_online.tof as gt
import gaps_online as go
import gaps_tof as gt_cxx
import gaps_online.db as db
import rust_dataclasses as rd
import re
from tqdm.notebook import tqdm
from glob import glob
from pathlib import Path
#import HErmes.fitting as fit

import dashi as d
import pylab as p
import numpy as np
d.visual()
import charmingbeauty.visual as vis
import charmingbeauty.layout as lo
vis.set_style_present()

# FIXME add DJANGO_ALLOW_ASYNC_UNSAFE=1 to setup.env
RUNPATH  = Path('/data1/nextcloud/cra_data/data/2023_nevis/tof/')
CALIPATH = RUNPATH / 'calibration' / '2024_03_12'

In [ ]:
print (CALIPATH)

In [ ]:
def skipped_events(events):
    n_skipped = 0
    last_evid = 0
    #print(len(events))
    evids = [k.mt_event.event_id for k in events]
    #print (min(evids))
    #print (max(evids))
    evids = [k for k in sorted(set(evids))]
    #print (len(evids))
    if evids[0] == 0:
        evids = evids[1:]
    last_evid = evids[0]
    #print (last_evid)
    #print (len(evids))
    #print (evids[:10])
    #raise
    for k in range(len(evids)):#, desc="Missing event search..."):
        if not last_evid in evids:
            n_skipped += 1
        last_evid += 1
        
    #for ev in events:
        #if ev.event_id != last_evid + 1:
        #if ev.header.event_id != last_evid + 1:
        #    if not last_evid == 0:
         #       n_skipped += 1
        #last_evid = ev.header.event_id
        #last_evid = ev.event_id
        #print (ev.event_id)
    return n_skipped

In [ ]:
runfiles = sorted((RUNPATH / '142').glob('*.tof.gaps'))
#runfiles = [k for k in runfiles][10:]
#print (runfiles)
#print (len(runfiles))
#raise
nfiles = 0
events = []
all_skipped = 0
last_evid   = 0
for rfile in tqdm(runfiles):
    #packets = gt.get_tofpackets(str(rfile), filter=go.cxx_api.PacketType.TofEventSummary)
    packets = gt.get_tofpackets(str(rfile), filter=go.cxx_api.PacketType.TofEvent)
    #print (packets)
    
    #raise
    #print (packets[1].payload)
    events = [go.cxx_api.TofEvent.from_tofpacket(k) for k in packets]
    #for k in events:
    #    print(k.header.event_id)
    #raise
    #print (events[0])
    #raise
    #events = [go.cxx_api.TofEventSummary.from_bytestream(k.payload, 0) for k in packets]
    #events = sorted(events, key = lambda x : x.header.event_id)
    #if not events:
    #    print (f"File {rfile} dos not contain any tof events!")
     #   continue
    #last_event = events[-1].header.event_id
    #last_event = events[-1].event_id
    #raise
    #if not last_evid == 0:
    #    if last_event - last_evid > 1:
    #        pass
            #print(f"We skipped {last_event-last_evid} events between the files!!!")
            #all_skipped += last_event - last_evid - 1
    #last_evid = last_event
    #n_skip = skipped_events(events[10000:20000])
    n_skip = skipped_events(events)
    #raise
    all_skipped += n_skip
    print (f'File {rfile} had {n_skip}/{len(events)} {100*n_skip/len(events):4.2f} % skipped event ids!')
    #break
    #events.extend([go.cxx_api.TofEvent.from_tofpacket(k) for k in packets])
    #nfiles += 1
    #if nfiles == 20:
    #    break

In [ ]:
print (len(events))
print (all_skipped)
#go.cxx_api.PacketType.

In [ ]:
events[0].mt_event

In [ ]:
hits = []
for k in tqdm(events):
    for n in k.rbevents:
        hits.extend(n.hits)

In [ ]:
print (len(hits))

In [ ]:
%matplotlib inline
events[0].rbevents[0].plot()

In [ ]:
%matplotlib inline

h = hits[0]
#print (h)
for h in hits:
    if h.charge_b != 0:
        print (h.charge_b)
        print (h)
        break

In [ ]:
peaks_a = [h.peak_a for h in hits]
peaks_b = [h.peak_b for h in hits]

hist_a = d.factory.hist1d(peaks_a, 100)
hist_b = d.factory.hist1d(peaks_b, 100)

hist_a.line()
hist_b.line()
p.show()